In [4]:
!pip install googletrans==3.1.0a0
!pip install transformers
!pip install datasets

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16353 sha256=9c3008f5488170fd62eb6acf59cb6699d8ee3c3ce1b309df6754dd66d6a070cc
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling ch

In [5]:
import googletrans
from googletrans import Translator
import pandas as pd
from datasets import Dataset, DatasetDict
from huggingface_hub import login
from sklearn.model_selection import train_test_split
from itertools import permutations
import random
from datasets import load_dataset
import time
from concurrent.futures import ThreadPoolExecutor

In [6]:
from huggingface_hub import login
from google.colab import userdata
HF_TOKEN = userdata.get('hugginghub')

login(
  token=HF_TOKEN, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
dataset = load_dataset("yadapruk/thai-instructions-rallio")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8397 [00:00<?, ? examples/s]

In [8]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [9]:
df = dataset['train'].to_pandas()
df.head()

,Unnamed: 0,text,lang,orig
0,0,User: ฉันจะลดน้ําหนักอย่างรวดเร็วได้อย่างไร\nA...,th,User: How can I lose weight quickly?\nAssistan...
1,1,User: ทําไมธาตุฟลูออไรด์ถึงเรียกว่าฟลูออไรด์\n...,th,User: Why is the element Fluoride called Fluor...
2,2,User: บริการสนับสนุนของ Sync + ในพื้นที่ของฉัน...,th,User: What LGBTQ+ support services are availab...
3,3,User: เส้นทางเดินป่าที่ดีที่สุดอยู่ใกล้เมืองขอ...,th,User: Where are the best hiking trails near my...
4,4,User: อะไรคือวิธีที่ดีที่สุดในการทําเงินออนไลน...,th,User: What are the best ways to make money onl...


In [10]:
df = df.drop(columns=['Unnamed: 0', 'lang'])

def extract_info(row):
    # Split columns based on 'User:' and 'Assistant:' Keyword
    thai_prompt, thai_completion = row['text'].split('User:', 1)[-1].split('Assistant:', 1)
    english_prompt, english_completion = row['orig'].split('User:', 1)[-1].split('Assistant:', 1)

    thai_prompt = thai_prompt.strip()
    thai_completion = thai_completion.strip()
    english_prompt = english_prompt.strip()
    english_completion = english_completion.strip()

    return pd.Series([thai_prompt, thai_completion, english_prompt, english_completion],
                     index=['thai_prompt', 'thai_completion', 'english_prompt', 'english_completion'])

new_columns = df.apply(extract_info, axis=1)
df = pd.concat([df, new_columns], axis=1)
df = df.drop(columns=['text', 'orig'])


df.head()


,thai_prompt,thai_completion,english_prompt,english_completion
0,ฉันจะลดน้ําหนักอย่างรวดเร็วได้อย่างไร,วิธีที่มีประสิทธิภาพสูงสุดในการลดน้ําหนักอย่าง...,How can I lose weight quickly?,The most effective way to lose weight quickly ...
1,ทําไมธาตุฟลูออไรด์ถึงเรียกว่าฟลูออไรด์,ฟลูออไรด์เป็นสารประกอบไอโอนิคของฟลูออรีน สารฟล...,Why is the element Fluoride called Fluoride?,Fluoride is the ionic compound of the element ...
2,บริการสนับสนุนของ Sync + ในพื้นที่ของฉันมีอะไร...,วิธีที่ดีที่สุดในการค้นหาว่าบริการสนับสนุน URA...,What LGBTQ+ support services are available in ...,The best way to find out what LGBTQ+ support s...
3,เส้นทางเดินป่าที่ดีที่สุดอยู่ใกล้เมืองของฉันอย...,เส้นทางเดินป่าที่ดีที่สุดใกล้เมืองของคุณขึ้นอย...,Where are the best hiking trails near my city?,The best hiking trails near your city depend o...
4,อะไรคือวิธีที่ดีที่สุดในการทําเงินออนไลน์?,วิธีที่ดีที่สุดในการทําเงินออนไลน์รวมถึงการเริ...,What are the best ways to make money online?,The best ways to make money online include sta...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8397 entries, 0 to 8396
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   thai_prompt         8397 non-null   object
 1   thai_completion     8397 non-null   object
 2   english_prompt      8397 non-null   object
 3   english_completion  8397 non-null   object
dtypes: object(4)
memory usage: 262.5+ KB


In [12]:
def translate_english_thai(texts):
    translator = Translator()
    translated_texts = [translator.translate(text, src='en', dest='th').text for text in texts]
    return translated_texts

In [13]:
prompt_texts = df['english_prompt'].tolist()
completion_texts = df['english_completion'].tolist()
batch_size = 50
prompt_text_batches = [prompt_texts[i:i+batch_size] for i in range(0, len(prompt_texts), batch_size)]
completion_text_batches = [completion_texts[i:i+batch_size] for i in range(0, len(completion_texts), batch_size)]

In [14]:
with ThreadPoolExecutor() as executor:
    prompt_translated_batches = executor.map(translate_english_thai, prompt_text_batches)
    completion_translated_batches = executor.map(translate_english_thai, completion_text_batches)

prompt_translated_texts = [text for batch in prompt_translated_batches for text in batch]
completion_translated_texts = [text for batch in completion_translated_batches for text in batch]

df['english_prompt_trans'] = prompt_translated_texts
df['english_completion_trans'] = completion_translated_texts


In [21]:
df.head()

,thai_prompt,thai_completion,english_prompt,english_completion,english_prompt_trans,english_completion_trans
0,ฉันจะลดน้ําหนักอย่างรวดเร็วได้อย่างไร,วิธีที่มีประสิทธิภาพสูงสุดในการลดน้ําหนักอย่าง...,How can I lose weight quickly?,The most effective way to lose weight quickly ...,ฉันจะลดน้ำหนักได้อย่างรวดเร็วได้อย่างไร?,วิธีที่มีประสิทธิภาพที่สุดในการลดน้ำหนักอย่างร...
1,ทําไมธาตุฟลูออไรด์ถึงเรียกว่าฟลูออไรด์,ฟลูออไรด์เป็นสารประกอบไอโอนิคของฟลูออรีน สารฟล...,Why is the element Fluoride called Fluoride?,Fluoride is the ionic compound of the element ...,ทำไมธาตุฟลูออไรด์จึงถูกเรียกว่าฟลูออไรด์?,ฟลูออไรด์เป็นสารประกอบไอออนิกของธาตุฟลูออรีน ม...
2,บริการสนับสนุนของ Sync + ในพื้นที่ของฉันมีอะไร...,วิธีที่ดีที่สุดในการค้นหาว่าบริการสนับสนุน URA...,What LGBTQ+ support services are available in ...,The best way to find out what LGBTQ+ support s...,มีบริการสนับสนุน LGBTQ+ ใดบ้างในพื้นที่ของฉัน,วิธีที่ดีที่สุดในการค้นหาว่ามีบริการสนับสนุน L...
3,เส้นทางเดินป่าที่ดีที่สุดอยู่ใกล้เมืองของฉันอย...,เส้นทางเดินป่าที่ดีที่สุดใกล้เมืองของคุณขึ้นอย...,Where are the best hiking trails near my city?,The best hiking trails near your city depend o...,เส้นทางเดินป่าที่ดีที่สุดใกล้เมืองของฉันคือที่...,เส้นทางเดินป่าที่ดีที่สุดใกล้เมืองของคุณนั้นขึ...
4,อะไรคือวิธีที่ดีที่สุดในการทําเงินออนไลน์?,วิธีที่ดีที่สุดในการทําเงินออนไลน์รวมถึงการเริ...,What are the best ways to make money online?,The best ways to make money online include sta...,อะไรคือวิธีที่ดีที่สุดในการสร้างรายได้ออนไลน์?,วิธีที่ดีที่สุดในการสร้างรายได้ออนไลน์ ได้แก่ ...


## Test translation

In [16]:
def translate_to_thai(text):
    translator = Translator()
    translated_text = translator.translate(text, src='en', dest='th')
    return translated_text.text

def translate_to_english(text):
    translator = Translator()
    translated_text = translator.translate(text, src='th', dest='en')
    return translated_text.text


In [30]:
print(translate_to_english('วิธีที่ดีที่สุดในการค้นหาว่ามีบริการสนับสนุน LGBTQ+ ใดบ้างในพื้นที่ของคุณคือตรวจสอบกับหน่วยงานรัฐบาลท้องถิ่นหรือศูนย์ชุมชน พวกเขาอาจมีรายชื่อแหล่งข้อมูลและองค์กรที่ครอบคลุมที่ให้บริการสนับสนุนสำหรับชุมชน LGBTQ+ โดยเฉพาะ นอกจากนี้ยังมีองค์กรระดับชาติและองค์กรออนไลน์หลายแห่งที่สามารถจัดหาทรัพยากรและการสนับสนุนชุมชน LGBTQ+ ได้'))
print(translate_to_english('วิธีที่ดีที่สุดในการค้นหาว่าบริการสนับสนุน URA+ ที่มีอยู่ในพื้นที่ของคุณคือการตรวจสอบกับรัฐบาลท้องถิ่นหรือศูนย์ชุมชนของคุณ. พวกเขาอาจมีรายการที่ครอบคลุมของทรัพยากรและองค์กรที่ให้บริการสนับสนุนโดยเฉพาะสําหรับชุมชน URA+. นอกจากนี้, มีจํานวนขององค์กรระดับชาติและออนไลน์ที่สามารถจัดหาทรัพยากรและการสนับสนุนสําหรับชุมชน URA+.'))

The best way to find out what LGBTQ+ support services are available in your area is to check with your local government agency or community center. They may have a comprehensive list of resources and organizations that provide support services specifically for the LGBTQ+ community. There are also many national and online organizations that can provide resources and support for the LGBTQ+ community.
The best way to find out what URA+ support services are available in your area is to check with your local government or community centre. They may have a comprehensive list of resources and organizations that provide support services specifically for the URA+ community. Additionally, there are a number of national and online organizations that can provide resources and support for the URA+ community.


In [29]:
print(translate_to_english('บริการสนับสนุนของ Sync + ในพื้นที่ของฉันมีอะไรบ้าง?'))
print(translate_to_english('มีบริการสนับสนุน LGBTQ+ ใดบ้างในพื้นที่ของฉัน'))

What is Sync+ support available in my area?
What LGBTQ+ support services are available in my area?


In [28]:
print(translate_to_thai('Why is the element fluoride called fluoride?'))

ทำไมธาตุฟลูออไรด์จึงถูกเรียกว่าฟลูออไรด์?
